# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

import warnings

warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
ls

categories.csv                     ML Pipeline Preparation.ipynb
DisasterResponse.db                ML Pipeline Preparation-zh.ipynb
ETL Pipeline Preparation.ipynb     model.sav
ETL Pipeline Preparation-zh.ipynb  Twitter-sentiment-self-drive-DFE.csv
messages.csv


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse", engine)

#X = df['message']
#Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1) 

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 12)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_train_pred = pipeline.predict(X_train)

# Print classification report on training data
print(classification_report(Y_train.values, y_train_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     15942
               request       1.00      0.92      0.96      3607
                 offer       1.00      0.76      0.86        91
           aid_related       1.00      0.97      0.98      8750
          medical_help       1.00      0.85      0.92      1684
      medical_products       1.00      0.84      0.91      1064
     search_and_rescue       1.00      0.79      0.88       582
              security       1.00      0.72      0.83       374
              military       1.00      0.86      0.92       690
                 water       1.00      0.92      0.96      1370
                  food       1.00      0.93      0.97      2335
               shelter       1.00      0.91      0.95      1862
              clothing       1.00      0.84      0.92       313
                 money       1.00      0.83      0.91       482
        missing_people       1.00      

In [10]:
y_pred = pipeline.predict(X_test)

# Print classification report on test data
print(classification_report(Y_test.values, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      3964
               request       0.79      0.38      0.52       867
                 offer       0.00      0.00      0.00        27
           aid_related       0.73      0.53      0.61      2110
          medical_help       0.69      0.10      0.17       400
      medical_products       0.52      0.04      0.08       249
     search_and_rescue       0.83      0.04      0.07       142
              security       0.00      0.00      0.00        97
              military       0.62      0.05      0.09       170
                 water       0.81      0.30      0.44       302
                  food       0.82      0.27      0.40       588
               shelter       0.72      0.13      0.22       452
              clothing       0.70      0.08      0.14        92
                 money       0.71      0.04      0.08       122
        missing_people       1.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {#'vect__min_df': [1, 5],
              #'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 3, 4]}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 25], 'clf__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# Get the prediction values from the grid search cross validator

y_train_pred = pipeline.predict(X_train)

# Print classification report on training data
print(classification_report(Y_train.values, y_train_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     15942
               request       1.00      0.92      0.96      3607
                 offer       1.00      0.76      0.86        91
           aid_related       1.00      0.97      0.98      8750
          medical_help       1.00      0.85      0.92      1684
      medical_products       1.00      0.84      0.91      1064
     search_and_rescue       1.00      0.79      0.88       582
              security       1.00      0.72      0.83       374
              military       1.00      0.86      0.92       690
                 water       1.00      0.92      0.96      1370
                  food       1.00      0.93      0.97      2335
               shelter       1.00      0.91      0.95      1862
              clothing       1.00      0.84      0.92       313
                 money       1.00      0.83      0.91       482
        missing_people       1.00      

In [13]:
y_pred = pipeline.predict(X_test)

# Print classification report on test data
print(classification_report(Y_test.values, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      3964
               request       0.79      0.38      0.52       867
                 offer       0.00      0.00      0.00        27
           aid_related       0.73      0.53      0.61      2110
          medical_help       0.69      0.10      0.17       400
      medical_products       0.52      0.04      0.08       249
     search_and_rescue       0.83      0.04      0.07       142
              security       0.00      0.00      0.00        97
              military       0.62      0.05      0.09       170
                 water       0.81      0.30      0.44       302
                  food       0.82      0.27      0.40       588
               shelter       0.72      0.13      0.22       452
              clothing       0.70      0.08      0.14        92
                 money       0.71      0.04      0.08       122
        missing_people       1.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 12)

model = pipeline.fit(X_train, Y_train)

y_pred = model.predict(X_test)

# Print classification report on test data
print(classification_report(Y_test.values, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.88      3964
               request       0.79      0.37      0.51       867
                 offer       0.00      0.00      0.00        27
           aid_related       0.75      0.51      0.61      2110
          medical_help       0.64      0.09      0.15       400
      medical_products       0.81      0.12      0.20       249
     search_and_rescue       0.68      0.12      0.20       142
              security       0.00      0.00      0.00        97
              military       0.43      0.06      0.10       170
                 water       0.82      0.24      0.37       302
                  food       0.83      0.29      0.42       588
               shelter       0.78      0.23      0.36       452
              clothing       0.75      0.13      0.22        92
                 money       1.00      0.05      0.09       122
        missing_people       1.00      

### 9. Export your model as a pickle file

In [16]:
pickle.dump(model, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [17]:
ls

categories.csv                     ML Pipeline Preparation.ipynb
DisasterResponse.db                ML Pipeline Preparation-zh.ipynb
ETL Pipeline Preparation.ipynb     model.sav
ETL Pipeline Preparation-zh.ipynb  Twitter-sentiment-self-drive-DFE.csv
messages.csv


In [19]:
import joblib as joblib

joblib.load("classifier.pkl")

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])